In [53]:
import os
import pandas as pd

In [54]:
# Read in raw data

# For XRF, ignore repeats and standards
xrf = pd.read_excel('raw/GAL-DV-21_XRF.xlsx',usecols='A:BB',index_col=0)

# For ICPMS, ignnore repeats and standards
icpms = pd.read_excel('raw/DV-21-ICPMS.xlsx',index_col=0,nrows=54)

# Make directory for processed data
os.makedirs('processed',exist_ok=True)

In [55]:
# Pull unnormalized major elements from XRF and transpose
majors_unnorm = xrf.iloc[5:15,:].T

# Pull normalized major elements from XRF and transpose to make samples index
majors_norm = xrf.iloc[19:29,:].T

# Pull LOI from XRF and transpose 
loi = xrf.iloc[16,:].T

# Remove extra space in majors_unnorm columns
new_cols = majors_unnorm.columns.str.replace(' ','')
new_cols_unnorm = [x+'_unnorm' for x in new_cols]
majors_unnorm_corr = majors_unnorm.copy()
majors_unnorm_corr.columns = new_cols_unnorm

# Remove extra space in majors_norm columns
new_cols = majors_norm.columns.str.replace(' ','')
majors_norm_corr = majors_norm.copy()
majors_norm_corr.columns = new_cols

# Pull XRF trace elements and transpose
xrf_trace = xrf.iloc[32:51,:].T

# Remove extra space in XRF Trace columns
new_cols = xrf_trace.columns.str[1:]
xrf_trace_corr = xrf_trace.copy()
xrf_trace_corr.columns = new_cols

# Remove 'ppm' from ICPMS columns
new_cols = icpms.columns.str[:-4]
icpms_corr = icpms.copy()
icpms_corr.columns  = new_cols

# Remove XRF data duplicated by ICPMS
common_cols = xrf_trace_corr.columns.intersection(icpms_corr.columns)
xrf_trace_culled = xrf_trace_corr.drop(common_cols,axis=1)

# Check what is in each file
print(majors_unnorm_corr.columns)
print(majors_norm_corr.columns)
print(loi.name)
print(xrf_trace_culled.columns)
print(icpms_corr.columns)
print(majors_norm_corr.index.equals(xrf_trace_culled.index))
print(xrf_trace_culled.index.equals(icpms_corr.index))

# Combine into single dataframe
data_organized = pd.concat([majors_unnorm_corr,majors_norm_corr,loi,xrf_trace_culled,icpms_corr],axis=1)
print(data_organized.columns)


 

Index(['SiO2_unnorm', 'TiO2_unnorm', 'Al2O3_unnorm', 'FeO*_unnorm',
       'MnO_unnorm', 'MgO_unnorm', 'CaO_unnorm', 'Na2O_unnorm', 'K2O_unnorm',
       'P2O5_unnorm'],
      dtype='object')
Index(['SiO2', 'TiO2', 'Al2O3', 'FeO*', 'MnO', 'MgO', 'CaO', 'Na2O', 'K2O',
       'P2O5'],
      dtype='object')
LOI %
Index(['Ni', 'Cr', 'V', 'Ga', 'Cu', 'Zn'], dtype='object')
Index(['La', 'Ce', 'Pr', 'Nd', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm',
       'Yb', 'Lu', 'Ba', 'Th', 'Nb', 'Y', 'Hf', 'Ta', 'U', 'Pb', 'Rb', 'Cs',
       'Sr', 'Sc', 'Zr'],
      dtype='object')
True
True
Index(['SiO2_unnorm', 'TiO2_unnorm', 'Al2O3_unnorm', 'FeO*_unnorm',
       'MnO_unnorm', 'MgO_unnorm', 'CaO_unnorm', 'Na2O_unnorm', 'K2O_unnorm',
       'P2O5_unnorm', 'SiO2', 'TiO2', 'Al2O3', 'FeO*', 'MnO', 'MgO', 'CaO',
       'Na2O', 'K2O', 'P2O5', 'LOI %', 'Ni', 'Cr', 'V', 'Ga', 'Cu', 'Zn', 'La',
       'Ce', 'Pr', 'Nd', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb',
       'Lu', 'Ba', 'Th', 'Nb', 'Y',

In [56]:
# Read in metadata
meta = pd.read_csv('metadata/gchm_smps_long.csv',index_col=0)

# Isolate columns of interest
meta_cols = ['Latitude','Longitude','Elevation','Rock_Type','Period','S_Domain']

# Create placeholder rows for 184 and 186 for now
meta.loc['G22184',:] = 0
meta.loc['G22186',:] = 0

# Fix the misspelled Khaishi
typo = meta[meta['S_Domain']=='Khashi'].index
print(typo)
meta.loc[typo,'S_Domain'] = 'Khaishi'

# Isolate rows that were actually run
meta_trimmed = meta.loc[data_organized.index,meta_cols]

# Add metadata to main table
data_final = data_organized.join(meta_trimmed)

Index(['G22045', 'G22046C'], dtype='object', name='Sample_Num')


In [57]:
# Write to CSV
data_final.to_csv('processed/data_22.csv')

In [58]:
# Combine with QE data
qe = pd.read_csv('processed/data_qe.csv',index_col=0)

data_combined = pd.concat([data_final,qe])
print(data_combined.index)
print(data_combined.columns)

data_combined.to_csv('processed/data.csv')

Index(['G22019', 'G22020', 'G22032A', 'G22033', 'G22034', 'G22035', 'G22036A',
       'G22037', 'G22038', 'G22039', 'G22041A', 'G22041C', 'G22042', 'G22044',
       'G22045', 'G22046C', 'G22049', 'G22051', 'G22053', 'G22063', 'G22065',
       'G22066', 'G22067', 'G22068A', 'G22068B', 'G22068C', 'G22069', 'G22076',
       'G22081', 'G22091', 'G22092', 'G22094', 'G22096', 'G22108A', 'G22109',
       'G22110', 'G22111', 'G22114', 'G22115B', 'G22118', 'G22119', 'G22120',
       'G22121', 'G22122', 'G22125', 'G22126', 'G22129', 'G22130', 'G22131',
       'G22141', 'G22173', 'G22184', 'G22186', 'C17069B', 'V17043A',
       '100311-3A'],
      dtype='object')
Index(['SiO2_unnorm', 'TiO2_unnorm', 'Al2O3_unnorm', 'FeO*_unnorm',
       'MnO_unnorm', 'MgO_unnorm', 'CaO_unnorm', 'Na2O_unnorm', 'K2O_unnorm',
       'P2O5_unnorm', 'SiO2', 'TiO2', 'Al2O3', 'FeO*', 'MnO', 'MgO', 'CaO',
       'Na2O', 'K2O', 'P2O5', 'LOI %', 'Ni', 'Cr', 'V', 'Ga', 'Cu', 'Zn', 'La',
       'Ce', 'Pr', 'Nd', 'Sm', 'Eu', 